In [8]:
%%writefile demo.py
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date

st.title("💰 Expense Tracker and Visualisation")

EXPENSE_FILE = "expenses.csv"
DEBT_FILE = "debts.csv"

# ------------------ Page Navigation ------------------
page = st.selectbox(
    "Select Page",
    ["Expense Tracker", "Debt Tracker", "Visualisations"]
)

# ------------------ Load Expense Data ------------------
try:
    exp_df = pd.read_csv(EXPENSE_FILE)
except FileNotFoundError:
    exp_df = pd.DataFrame(columns=["Date", "Category", "Amount", "Description"])

exp_dates = exp_df["Date"].tolist()
exp_categories = exp_df["Category"].tolist()
exp_amounts = exp_df["Amount"].tolist()
exp_descriptions = exp_df["Description"].tolist()

# -------- SORT EXPENSE DATA BY DATE USING sorted() WITHOUT zip() --------
indices = list(range(len(exp_dates)))
sorted_indices = sorted(indices, key=lambda x: exp_dates[x])

exp_dates = [exp_dates[i] for i in sorted_indices]
exp_categories = [exp_categories[i] for i in sorted_indices]
exp_amounts = [exp_amounts[i] for i in sorted_indices]
exp_descriptions = [exp_descriptions[i] for i in sorted_indices]

exp_df = pd.DataFrame({
    "Date": exp_dates,
    "Category": exp_categories,
    "Amount": exp_amounts,
    "Description": exp_descriptions
})

# ------------------ Load Debt Data ------------------
try:
    debt_df = pd.read_csv(DEBT_FILE)
except FileNotFoundError:
    debt_df = pd.DataFrame(
        columns=["Date", "Person", "Amount", "Type", "Note", "Status"]
    )

debt_dates = debt_df["Date"].tolist()
debt_people = debt_df["Person"].tolist()
debt_amounts = debt_df["Amount"].tolist()
debt_types = debt_df["Type"].tolist()
debt_notes = debt_df["Note"].tolist()
debt_status = debt_df["Status"].tolist()

# -------- SORT DEBT DATA BY DATE USING sorted() WITHOUT zip() --------
indices_d = list(range(len(debt_dates)))
sorted_indices_d = sorted(indices_d, key=lambda x: debt_dates[x])

debt_dates = [debt_dates[i] for i in sorted_indices_d]
debt_people = [debt_people[i] for i in sorted_indices_d]
debt_amounts = [debt_amounts[i] for i in sorted_indices_d]
debt_types = [debt_types[i] for i in sorted_indices_d]
debt_notes = [debt_notes[i] for i in sorted_indices_d]
debt_status = [debt_status[i] for i in sorted_indices_d]

debt_df = pd.DataFrame({
    "Date": debt_dates,
    "Person": debt_people,
    "Amount": debt_amounts,
    "Type": debt_types,
    "Note": debt_notes,
    "Status": debt_status
})

# ====================================================
# ================= EXPENSE PAGE =====================
# ====================================================
if page == "Expense Tracker":

    st.sidebar.header("➕ Add Expense")

    expense_date = st.sidebar.date_input("Date", date.today())
    category = st.sidebar.selectbox(
        "Category",
        ["Food", "Transport", "Entertainment", "Utilities", "Other"]
    )
    amount = st.sidebar.number_input("Amount", min_value=0.0)
    description = st.sidebar.text_input("Description")

    if st.sidebar.button("Add Expense"):
        if amount <= 0:
            st.sidebar.error("Amount must be greater than zero")
        else:
            exp_dates.append(str(expense_date))
            exp_categories.append(category)
            exp_amounts.append(amount)
            exp_descriptions.append(description)

            new_df = pd.DataFrame({
                "Date": exp_dates,
                "Category": exp_categories,
                "Amount": exp_amounts,
                "Description": exp_descriptions
            })
            new_df.to_csv(EXPENSE_FILE, index=False)
            st.sidebar.success("Expense Added Successfully!")

    st.subheader("📊 Summary")

    if len(exp_amounts) > 0:
        total = 0
        for a in exp_amounts:
            total += a

        st.write("**Total Spent:** ₹", total)
        st.write("**Average Expense:** ₹", round(total / len(exp_amounts), 2))
    else:
        st.info("No expenses recorded yet.")

    # ---------------- MONTHLY BUDGET PROGRESS ----------------
    st.subheader("🎯 Monthly Budget Progress")

    # User selects month
    month_input = st.date_input("Choose Month for Budget", date.today())
    selected_month = str(month_input)[:7]  # YYYY-MM

    budget = st.number_input("Set Monthly Budget (₹)", min_value=0.0)

    if budget > 0:
        monthly_total = 0

        for i in range(len(exp_dates)):
            if exp_dates[i][:7] == selected_month:
                monthly_total += exp_amounts[i]

        st.write(f"Spent in {selected_month}: ₹{monthly_total}")
        progress = int((monthly_total / budget) * 100)
        if progress > 100:
            progress = 100

        st.progress(progress)

    st.subheader("📋 Expense History (Oldest → Newest)")
    st.dataframe(exp_df)

# ====================================================
# ================= DEBT PAGE ========================
# ====================================================
if page == "Debt Tracker":

    st.sidebar.header("➕ Add Debt")

    debt_date = st.sidebar.date_input("Date", date.today())
    person = st.sidebar.text_input("Person Name")
    debt_amount = st.sidebar.number_input("Amount", min_value=0.0)
    debt_type = st.sidebar.selectbox(
        "Type",
        ["I Owe", "Owed To Me"]
    )
    note = st.sidebar.text_input("Note")

    if st.sidebar.button("Add Debt"):
        if debt_amount <= 0 or person == "":
            st.sidebar.error("Enter valid details")
        else:
            debt_dates.append(str(debt_date))
            debt_people.append(person)
            debt_amounts.append(debt_amount)
            debt_types.append(debt_type)
            debt_notes.append(note)
            debt_status.append("Unpaid")

            new_debt_df = pd.DataFrame({
                "Date": debt_dates,
                "Person": debt_people,
                "Amount": debt_amounts,
                "Type": debt_types,
                "Note": debt_notes,
                "Status": debt_status
            })
            new_debt_df.to_csv(DEBT_FILE, index=False)
            st.sidebar.success("Debt Added Successfully!")

    st.subheader("📊 Debt Summary (Unpaid Only)")

    total_owe = 0
    total_get = 0

    for i in range(len(debt_amounts)):
        if debt_status[i] == "Unpaid":
            if debt_types[i] == "I Owe":
                total_owe += debt_amounts[i]
            else:
                total_get += debt_amounts[i]

    st.error(f"💸 You Owe: ₹{total_owe}")
    st.success(f"💰 Owed To You: ₹{total_get}")
    st.info(f"📌 Net Balance: ₹{total_get - total_owe}")

    st.subheader("✅ Mark Debt as Paid")

    unpaid = []

    for i in range(len(debt_people)):
        if debt_status[i] == "Unpaid":
            unpaid.append(f"{i} - {debt_people[i]} ₹{debt_amounts[i]}")

    if len(unpaid) == 0:
        st.success("All debts cleared 🎉")
    else:
        selected = st.selectbox("Select Debt", unpaid)

        if st.button("Mark as Paid"):
            idx = int(selected.split(" - ")[0])
            debt_status[idx] = "Paid"

            updated_df = pd.DataFrame({
                "Date": debt_dates,
                "Person": debt_people,
                "Amount": debt_amounts,
                "Type": debt_types,
                "Note": debt_notes,
                "Status": debt_status
            })
            updated_df.to_csv(DEBT_FILE, index=False)

            st.success("Debt marked as Paid ✅")

    st.subheader("📋 Debt History")
    st.dataframe(debt_df)

# ====================================================
# ================= Visualisation PAGE ===============
# ====================================================
if page == "Visualisations":

    st.header("📈 Expense Visualisations")

    if len(exp_amounts) == 0:
        st.warning("No data available.")
    else:
        # -------- DAILY HISTOGRAM --------
        st.subheader("📊 Daily Expense Distribution (Histogram)")

        plt.figure(figsize=(6, 4))
        plt.hist(exp_amounts, bins=10)
        plt.xlabel("Expense Amount")
        plt.ylabel("Frequency")
        plt.title("Daily Expense Histogram")
        st.pyplot(plt)
        plt.clf()

        # -------- MONTHLY LINE GRAPH --------
        st.subheader("📈 Monthly Expense Trend")

        monthly_data = {}

        for i in range(len(exp_dates)):
            month = exp_dates[i][:7]
            if month in monthly_data:
                monthly_data[month] += exp_amounts[i]
            else:
                monthly_data[month] = exp_amounts[i]

        months = []
        totals = []

        for m in monthly_data:
            months.append(m)
            totals.append(monthly_data[m])

        plt.figure(figsize=(6, 4))
        plt.plot(months, totals, marker="o")
        plt.xlabel("Month")
        plt.ylabel("Total Spent")
        plt.title("Monthly Expense Line Graph")
        st.pyplot(plt)
        plt.clf()

        # -------- CATEGORY PIE CHART --------
        st.subheader("🥧 Category-wise Expense Distribution")

        category_data = {}

        for i in range(len(exp_categories)):
            if exp_categories[i] in category_data:
                category_data[exp_categories[i]] += exp_amounts[i]
            else:
                category_data[exp_categories[i]] = exp_amounts[i]

        labels = []
        values = []

        for c in category_data:
            labels.append(c)
            values.append(category_data[c])

        plt.figure(figsize=(5, 4))
        plt.pie(values, labels=labels, autopct="%1.1f%%")
        plt.title("Expenses by Category")
        st.pyplot(plt)
        plt.clf()


Overwriting demo.py


# 💰 Expense Tracker Application - Line by Line Explanation

## Overview
This Streamlit application tracks personal expenses and debts with visualization capabilities. It uses CSV files for data persistence and provides an interactive web interface.

---

## Import Statements

```python
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date
```

**Line 1:** Imports Streamlit library for creating the web application interface  
**Line 2:** Imports Pandas for data manipulation and CSV file handling  
**Line 3:** Imports Matplotlib for creating data visualizations (charts and graphs)  
**Line 4:** Imports the date class from datetime module to work with dates

---

## Application Title and File Configuration

```python
st.title("💰 Expense Tracker and Visualisation")

EXPENSE_FILE = "expenses.csv"
DEBT_FILE = "debts.csv"
```

**Line 6:** Sets the main title of the web application with an emoji  
**Line 8:** Defines constant for the expense data CSV filename  
**Line 9:** Defines constant for the debt data CSV filename

---

## Page Navigation

```python
page = st.selectbox(
    "Select Page",
    ["Expense Tracker", "Debt Tracker", "Visualisations"]
)
```

**Lines 11-14:** Creates a dropdown menu allowing users to navigate between three pages:
- Expense Tracker (for adding and viewing expenses)
- Debt Tracker (for managing money owed/borrowed)
- Visualisations (for viewing charts and graphs)

---

## Loading Expense Data

```python
try:
    exp_df = pd.read_csv(EXPENSE_FILE)
except FileNotFoundError:
    exp_df = pd.DataFrame(columns=["Date", "Category", "Amount", "Description"])
```

**Line 17:** Attempts to read the expenses CSV file  
**Line 18:** Loads existing expense data into a DataFrame  
**Line 19:** Catches error if the file doesn't exist (first time running)  
**Line 20:** Creates an empty DataFrame with the required column structure

---

## Extracting Expense Data into Lists

```python
exp_dates = exp_df["Date"].tolist()
exp_categories = exp_df["Category"].tolist()
exp_amounts = exp_df["Amount"].tolist()
exp_descriptions = exp_df["Description"].tolist()
```

**Line 22:** Converts the Date column into a Python list  
**Line 23:** Converts the Category column into a Python list  
**Line 24:** Converts the Amount column into a Python list  
**Line 25:** Converts the Description column into a Python list

*Why use lists?* Lists allow for easier manipulation and sorting of data using basic Python operations.

---

## Sorting Expense Data by Date

```python
indices = list(range(len(exp_dates)))
sorted_indices = sorted(indices, key=lambda x: exp_dates[x])
```

**Line 28:** Creates a list of index numbers [0, 1, 2, 3, ...] for all expense entries  
**Line 29:** Sorts these indices based on their corresponding dates using a lambda function

The `lambda x: exp_dates[x]` means: "for each index x, look up the date at position x"

```python
exp_dates = [exp_dates[i] for i in sorted_indices]
exp_categories = [exp_categories[i] for i in sorted_indices]
exp_amounts = [exp_amounts[i] for i in sorted_indices]
exp_descriptions = [exp_descriptions[i] for i in sorted_indices]
```

**Lines 31-34:** Reorder all four lists using the sorted indices via list comprehension  
This ensures all expense data is arranged chronologically from oldest to newest.

---

## Reconstructing Sorted Expense DataFrame

```python
exp_df = pd.DataFrame({
    "Date": exp_dates,
    "Category": exp_categories,
    "Amount": exp_amounts,
    "Description": exp_descriptions
})
```

**Lines 36-41:** Creates a new DataFrame with the sorted data, ensuring all expenses are in chronological order for display and analysis.

---

## Loading Debt Data

```python
try:
    debt_df = pd.read_csv(DEBT_FILE)
except FileNotFoundError:
    debt_df = pd.DataFrame(
        columns=["Date", "Person", "Amount", "Type", "Note", "Status"]
    )
```

**Line 44:** Attempts to read the debts CSV file  
**Line 45:** Loads existing debt data into a DataFrame  
**Line 46:** Catches error if the file doesn't exist  
**Lines 47-49:** Creates an empty DataFrame with six columns for debt tracking

---

## Extracting Debt Data into Lists

```python
debt_dates = debt_df["Date"].tolist()
debt_people = debt_df["Person"].tolist()
debt_amounts = debt_df["Amount"].tolist()
debt_types = debt_df["Type"].tolist()
debt_notes = debt_df["Note"].tolist()
debt_status = debt_df["Status"].tolist()
```

**Lines 51-56:** Converts each column of the debt DataFrame into separate Python lists for manipulation.

---

## Sorting Debt Data by Date

```python
indices_d = list(range(len(debt_dates)))
sorted_indices_d = sorted(indices_d, key=lambda x: debt_dates[x])

debt_dates = [debt_dates[i] for i in sorted_indices_d]
debt_people = [debt_people[i] for i in sorted_indices_d]
debt_amounts = [debt_amounts[i] for i in sorted_indices_d]
debt_types = [debt_types[i] for i in sorted_indices_d]
debt_notes = [debt_notes[i] for i in sorted_indices_d]
debt_status = [debt_status[i] for i in sorted_indices_d]
```

**Line 59:** Creates index list for debt entries  
**Line 60:** Sorts indices by date  
**Lines 62-67:** Reorders all debt lists chronologically using the same technique as expenses

---

## Reconstructing Sorted Debt DataFrame

```python
debt_df = pd.DataFrame({
    "Date": debt_dates,
    "Person": debt_people,
    "Amount": debt_amounts,
    "Type": debt_types,
    "Note": debt_notes,
    "Status": debt_status
})
```

**Lines 69-76:** Creates a new DataFrame with sorted debt data in chronological order.

---

## EXPENSE TRACKER PAGE

### Sidebar Form for Adding Expenses

```python
if page == "Expense Tracker":
    
    st.sidebar.header("➕ Add Expense")
    
    expense_date = st.sidebar.date_input("Date", date.today())
    category = st.sidebar.selectbox(
        "Category",
        ["Food", "Transport", "Entertainment", "Utilities", "Other"]
    )
    amount = st.sidebar.number_input("Amount", min_value=0.0)
    description = st.sidebar.text_input("Description")
```

**Line 82:** Checks if user selected the Expense Tracker page  
**Line 84:** Creates a sidebar section header  
**Line 86:** Date picker widget, defaulting to today's date  
**Lines 87-90:** Dropdown menu for selecting expense category  
**Line 91:** Number input field for amount (cannot be negative)  
**Line 92:** Text input field for optional description

---

### Processing New Expense Entry

```python
if st.sidebar.button("Add Expense"):
    if amount <= 0:
        st.sidebar.error("Amount must be greater than zero")
    else:
        exp_dates.append(str(expense_date))
        exp_categories.append(category)
        exp_amounts.append(amount)
        exp_descriptions.append(description)
```

**Line 94:** Creates button; code runs when clicked  
**Line 95:** Validates that amount is positive  
**Line 96:** Shows error message if validation fails  
**Lines 98-101:** Appends new expense data to all four lists

---

### Saving Expense to CSV

```python
        new_df = pd.DataFrame({
            "Date": exp_dates,
            "Category": exp_categories,
            "Amount": exp_amounts,
            "Description": exp_descriptions
        })
        new_df.to_csv(EXPENSE_FILE, index=False)
        st.sidebar.success("Expense Added Successfully!")
```

**Lines 103-108:** Creates DataFrame from updated lists  
**Line 109:** Saves DataFrame to CSV file (index=False prevents row numbers)  
**Line 110:** Displays success message in sidebar

---

### Expense Summary Statistics

```python
st.subheader("📊 Summary")

if len(exp_amounts) > 0:
    total = 0
    for a in exp_amounts:
        total += a
    
    st.write("**Total Spent:** ₹", total)
    st.write("**Average Expense:** ₹", round(total / len(exp_amounts), 2))
else:
    st.info("No expenses recorded yet.")
```

**Line 112:** Creates a subheading for summary section  
**Line 114:** Checks if any expenses exist  
**Line 115:** Initializes total counter  
**Lines 116-117:** Loops through amounts and calculates sum  
**Line 119:** Displays total expenses in rupees  
**Line 120:** Calculates and displays average (rounded to 2 decimals)  
**Lines 121-122:** Shows info message if no data exists

---

### Monthly Budget Progress Tracker

```python
st.subheader("🎯 Monthly Budget Progress")

month_input = st.date_input("Choose Month for Budget", date.today())
selected_month = str(month_input)[:7]  # YYYY-MM

budget = st.number_input("Set Monthly Budget (₹)", min_value=0.0)
```

**Line 125:** Creates subheading for budget section  
**Line 128:** Date picker for selecting month  
**Line 129:** Extracts year-month portion (e.g., "2026-01") from full date

**Line 131:** Number input for setting monthly budget limit

---

### Calculating Monthly Spending

```python
if budget > 0:
    monthly_total = 0
    
    for i in range(len(exp_dates)):
        if exp_dates[i][:7] == selected_month:
            monthly_total += exp_amounts[i]
    
    st.write(f"Spent in {selected_month}: ₹{monthly_total}")
    progress = int((monthly_total / budget) * 100)
    if progress > 100:
        progress = 100
    
    st.progress(progress)
```

**Line 133:** Only calculates if user entered a budget  
**Line 134:** Initializes monthly total counter  
**Line 136:** Loops through all expense dates  
**Line 137:** Checks if expense belongs to selected month (compares YYYY-MM)  
**Line 138:** Adds matching expenses to monthly total  
**Line 140:** Displays amount spent in the selected month  
**Line 141:** Calculates percentage of budget used  
**Lines 142-143:** Caps progress at 100% for display purposes  
**Line 145:** Shows visual progress bar

---

### Displaying Expense History

```python
st.subheader("📋 Expense History (Oldest → Newest)")
st.dataframe(exp_df)
```

**Line 147:** Creates subheading indicating chronological order  
**Line 148:** Displays the full expense DataFrame as an interactive table

---

## DEBT TRACKER PAGE

### Sidebar Form for Adding Debts

```python
if page == "Debt Tracker":
    
    st.sidebar.header("➕ Add Debt")
    
    debt_date = st.sidebar.date_input("Date", date.today())
    person = st.sidebar.text_input("Person Name")
    debt_amount = st.sidebar.number_input("Amount", min_value=0.0)
    debt_type = st.sidebar.selectbox(
        "Type",
        ["I Owe", "Owed To Me"]
    )
    note = st.sidebar.text_input("Note")
```

**Line 154:** Checks if user selected Debt Tracker page  
**Line 156:** Creates sidebar header  
**Line 158:** Date picker for debt transaction  
**Line 159:** Text input for person's name  
**Line 160:** Number input for debt amount  
**Lines 161-164:** Dropdown to specify debt direction  
**Line 165:** Optional note field

---

### Processing New Debt Entry

```python
if st.sidebar.button("Add Debt"):
    if debt_amount <= 0 or person == "":
        st.sidebar.error("Enter valid details")
    else:
        debt_dates.append(str(debt_date))
        debt_people.append(person)
        debt_amounts.append(debt_amount)
        debt_types.append(debt_type)
        debt_notes.append(note)
        debt_status.append("Unpaid")
```

**Line 167:** Creates add button  
**Line 168:** Validates amount is positive and name is not empty  
**Line 169:** Shows error if validation fails  
**Lines 171-175:** Appends new debt details to respective lists  
**Line 176:** Sets initial status as "Unpaid"

---

### Saving Debt to CSV

```python
        new_debt_df = pd.DataFrame({
            "Date": debt_dates,
            "Person": debt_people,
            "Amount": debt_amounts,
            "Type": debt_types,
            "Note": debt_notes,
            "Status": debt_status
        })
        new_debt_df.to_csv(DEBT_FILE, index=False)
        st.sidebar.success("Debt Added Successfully!")
```

**Lines 178-185:** Creates DataFrame from updated debt lists  
**Line 186:** Saves to CSV file  
**Line 187:** Displays success message

---

### Debt Summary Calculations

```python
st.subheader("📊 Debt Summary (Unpaid Only)")

total_owe = 0
total_get = 0

for i in range(len(debt_amounts)):
    if debt_status[i] == "Unpaid":
        if debt_types[i] == "I Owe":
            total_owe += debt_amounts[i]
        else:
            total_get += debt_amounts[i]
```

**Line 189:** Creates subheading for debt summary  
**Lines 191-192:** Initialize counters for money owed and money to receive  
**Line 194:** Loops through all debt entries  
**Line 195:** Only processes unpaid debts  
**Lines 196-197:** Adds to "I owe" total  
**Lines 198-199:** Adds to "owed to me" total

---

### Displaying Debt Summary

```python
st.error(f"💸 You Owe: ₹{total_owe}")
st.success(f"💰 Owed To You: ₹{total_get}")
st.info(f"📌 Net Balance: ₹{total_get - total_owe}")
```

**Line 201:** Shows money you owe in red (error styling)  
**Line 202:** Shows money owed to you in green (success styling)  
**Line 203:** Shows net balance (positive = you'll receive, negative = you'll pay)

---

### Marking Debts as Paid

```python
st.subheader("✅ Mark Debt as Paid")

unpaid = []

for i in range(len(debt_people)):
    if debt_status[i] == "Unpaid":
        unpaid.append(f"{i} - {debt_people[i]} ₹{debt_amounts[i]}")
```

**Line 205:** Creates subheading for payment section  
**Line 207:** Initializes empty list for unpaid debts  
**Line 209:** Loops through all debt entries  
**Line 210:** Filters for unpaid debts only  
**Line 211:** Creates formatted string with index, person name, and amount

---

### Debt Payment Interface

```python
if len(unpaid) == 0:
    st.success("All debts cleared 🎉")
else:
    selected = st.selectbox("Select Debt", unpaid)
    
    if st.button("Mark as Paid"):
        idx = int(selected.split(" - ")[0])
        debt_status[idx] = "Paid"
```

**Lines 213-214:** Shows congratulations message if all debts paid  
**Line 216:** Creates dropdown with list of unpaid debts  
**Line 218:** Creates payment confirmation button  
**Line 219:** Extracts the index number from the selected option  
**Line 220:** Updates status to "Paid" in the list

---

### Saving Updated Debt Status

```python
        updated_df = pd.DataFrame({
            "Date": debt_dates,
            "Person": debt_people,
            "Amount": debt_amounts,
            "Type": debt_types,
            "Note": debt_notes,
            "Status": debt_status
        })
        updated_df.to_csv(DEBT_FILE, index=False)
        
        st.success("Debt marked as Paid ✅")
```

**Lines 222-229:** Creates DataFrame with updated status  
**Line 230:** Saves changes to CSV file  
**Line 232:** Displays confirmation message

---

### Displaying Debt History

```python
st.subheader("📋 Debt History")
st.dataframe(debt_df)
```

**Line 234:** Creates subheading  
**Line 235:** Shows complete debt history in table format

---

## VISUALISATIONS PAGE

### Daily Expense Histogram

```python
if page == "Visualisations":
    
    st.header("📈 Expense Visualisations")
    
    if len(exp_amounts) == 0:
        st.warning("No data available.")
    else:
        st.subheader("📊 Daily Expense Distribution (Histogram)")
        
        plt.figure(figsize=(6, 4))
        plt.hist(exp_amounts, bins=10)
        plt.xlabel("Expense Amount")
        plt.ylabel("Frequency")
        plt.title("Daily Expense Histogram")
        st.pyplot(plt)
        plt.clf()
```

**Line 241:** Checks if Visualisations page selected  
**Line 243:** Creates page header  
**Lines 245-246:** Shows warning if no data exists  
**Line 248:** Creates subheading for histogram  
**Line 250:** Creates figure with specified size  
**Line 251:** Creates histogram with 10 bins (groups expense ranges)  
**Lines 252-254:** Adds axis labels and title  
**Line 255:** Displays the plot in Streamlit  
**Line 256:** Clears the plot to prepare for next graph

---

### Monthly Expense Trend Line Graph

```python
        st.subheader("📈 Monthly Expense Trend")
        
        monthly_data = {}
        
        for i in range(len(exp_dates)):
            month = exp_dates[i][:7]
            if month in monthly_data:
                monthly_data[month] += exp_amounts[i]
            else:
                monthly_data[month] = exp_amounts[i]
```

**Line 259:** Creates subheading  
**Line 261:** Initializes empty dictionary for aggregation  
**Line 263:** Loops through all expense dates  
**Line 264:** Extracts month (YYYY-MM format)  
**Lines 265-266:** Adds to existing month total if month exists  
**Lines 267-268:** Creates new month entry if first time seeing this month

---

### Preparing Data for Line Graph

```python
        months = []
        totals = []
        
        for m in monthly_data:
            months.append(m)
            totals.append(monthly_data[m])
```

**Lines 270-271:** Initialize lists for x-axis and y-axis data  
**Line 273:** Loops through the monthly dictionary  
**Line 274:** Adds month to x-axis list  
**Line 275:** Adds corresponding total to y-axis list

---

### Creating Line Graph

```python
        plt.figure(figsize=(6, 4))
        plt.plot(months, totals, marker="o")
        plt.xlabel("Month")
        plt.ylabel("Total Spent")
        plt.title("Monthly Expense Line Graph")
        st.pyplot(plt)
        plt.clf()
```

**Line 277:** Creates new figure  
**Line 278:** Plots line graph with circular markers at each point  
**Lines 279-281:** Adds labels and title  
**Line 282:** Displays in Streamlit  
**Line 283:** Clears plot

---

### Category-wise Pie Chart

```python
        st.subheader("🥧 Category-wise Expense Distribution")
        
        category_data = {}
        
        for i in range(len(exp_categories)):
            if exp_categories[i] in category_data:
                category_data[exp_categories[i]] += exp_amounts[i]
            else:
                category_data[exp_categories[i]] = exp_amounts[i]
```

**Line 286:** Creates subheading  
**Line 288:** Initializes dictionary for category totals  
**Line 290:** Loops through all expense categories  
**Lines 291-292:** Adds amount to existing category total  
**Lines 293-294:** Creates new category entry with amount

---

### Preparing Pie Chart Data

```python
        labels = []
        values = []
        
        for c in category_data:
            labels.append(c)
            values.append(category_data[c])
```

**Lines 296-297:** Initialize lists for labels and values  
**Line 299:** Loops through category dictionary  
**Line 300:** Adds category name to labels  
**Line 301:** Adds category total to values

---

### Creating Pie Chart

```python
        plt.figure(figsize=(5, 4))
        plt.pie(values, labels=labels, autopct="%1.1f%%")
        plt.title("Expenses by Category")
        st.pyplot(plt)
        plt.clf()
```

**Line 303:** Creates figure  
**Line 304:** Creates pie chart with percentage labels (1 decimal place)  
**Line 305:** Adds title  
**Line 306:** Displays in Streamlit  
**Line 307:** Clears plot for next use

---

## Summary

This application provides a complete expense and debt tracking solution with:

- **Persistent data storage** using CSV files
- **User-friendly interface** with Streamlit widgets
- **Data validation** to ensure data quality
- **Multiple visualizations** to understand spending patterns
- **Budget tracking** to monitor monthly spending
- **Debt management** to track money owed and borrowed

The code demonstrates key concepts including file I/O, data manipulation with Pandas, list operations, conditional logic, and data visualization with Matplotlib.